## 영역 함수 ( Scope Functions )
- 객체의 컨텍스트 안에서 코드 블록을 실행할 수 있게 해주는 함수들.
- let, run, with, apply, also 등이 있다.

## run
- 확장 람다를 받는 확장 함수이며, 람다의 결과를 돌려준다.
- run 함수는 기본적으로 이렇게 생겼다. 모든 객체에 run을 붙일 수 있다.
    - ```kotlin
      inline fun <T, R> T.run(block: T.() -> R): R = block()
      ```
- 람다 내부에서 this로 수신객체 접근
- 람다 마지막 표현식이 반환된다.

In [ ]:
var result: Int = "Hello".run {
    println(this) // this = "Hello"
    this.length // 반환
}

// 객체 초기화 , 설정
data class Person (
    var name: String = "",
    var age: Int = 0
)

val person: Person = Person().run {
    name = "이광호"
    age = 20
    this // 반환
}

// 계산 후 결과 반환
val message = StringBuilder().run {
    append("Hello, ")
    append("Kotlin")
    toString()
}

println(message)

// null 세이프 같이 사용 가능
val name: String? = null
val greeting = name?.run {
    "Hello, $this"
} ?: "Hello, Stranger"

println(greeting)

// 일반 함수 형태로도 사용 가능
val greeting2 = run {
    "Hello, Kotlin"
}

## with
- 매개변수와 확장 람다를 받는 일반 함수. 확장 함수 아니다.
- with의 코드를 보면, receiver를 매개변수로 하나 받고, block 확장 람다도 받는다. run 과 동일하게 마지막 표현식을 반환한다.
    - `inline fun <T, R> with(receiver: T, block: T.() -> R): R`
- run과 다르게 null safe 하지는 않다.


In [4]:
// 일반 사용법
val result = with("Hello") {
    println(this)
    this.length
}

println(result)

// 객체 프로퍼티 / 메서드 접근
data class Person(
    val name: String,
    val age: Int,
)

val person = Person("광호", 20)

val result2 = with(person) {
    println(name)
    println(age)
    age + 10
}

println(result2)

val string: String? = "hello"

// run과 달리 안전 호출 연산자는 사용 불가능하다.
val result3 = with(string) {
    length
}

val result4 = string?.run {
    length
}

println(result3)

Hello
5
광호
20
30
5


## let
- run과 비슷하게 생겼다.
    `inline fun <T, R> T.let(block: (T) -> R): R`
- run과 동일하지만 객체 지시자를 it을 사용한다.
- run과 마찬가지로 null safe 연산자 사용 가능하다.

In [9]:
val result = "hello".let {
    println(it)
    it.length
}

val result2 = "hello".let { r ->
    println(r)
    r.length
}

val string: String? = null

val result3 = string?.let {
    it.length
} ?: "null 임"

println(result3)

hello
hello
null 임


## apply
- 함수 내부에서 this로 접근
- 수신 객체 자체를 반환, Unit을 반환한다.
- `inline fun <T> T.apply(block: T.() -> Unit): T`
- 주 사용처는 객체 초기화. 그리고 빌터 패턴 구현도 가능하다.

In [11]:
val result = "hello".apply {
    println(this)
    length // length가 아니라 객체가 반환된다.
}

println(result)

data class Person (
    var name: String = "",
    var age: Int = 0
)

val person = Person().apply {
    name = "이광호"
    age = 20
}

// 빌더 패턴
class DatabaseConfig {
    var host: String = "localhost"
    var port: Int = 5432
    var username: String = ""
    var password: String = ""
    var database: String = ""

    fun build(): String {
        return "jdbc:postgresql://$host:$port/$database"
    }
}

val config = DatabaseConfig().apply {
    host = "db.example.com"
    port = 5432
    username = "admin"
    password = "secret"
    database = "myapp"
}.build()


hello
hello
jdbc:postgresql://db.example.com:5432/myapp


## also
- 함수 내부에서 it으로 접근
- `inline fun <T> T.also(block: (T) -> Unit): T`
- apply와 동일하게 수신 객체를 반환


In [12]:
val result = "hello".also {
    println(it)
    it.length
}
println(result)

hello
hello
